In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import ensemble
from sklearn.inspection import permutation_importance

In [2]:
df = pd.read_csv('wc-wo-outliers.csv')

In [3]:
y = df['results']
X = df.drop(columns=['results'])

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.20, random_state=1)

In [4]:
params = {
    "n_estimators": 500,
    "max_depth": 400,
    "min_samples_split": 4,
    "learning_rate": 0.0159,
    "loss": "squared_error",
    "max_features": 14,
    "min_weight_fraction_leaf": 0.1,
}

In [5]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

# mse = mean_squared_error(y_test, reg.predict(X_test))
predictions = reg.predict(X_train)

print('GRADIENT BOOSTING REGRESSION')
# print("The mean squared error (MSE) on test set: {:.4f}".format(mse))


print('RMSE: '+str(np.sqrt(mean_squared_error(y_train, predictions))))
print('R Squared: '+str(r2_score(y_train, predictions)))
print('MSE : '+str(mean_squared_error(y_train, predictions)))
print('MAE : '+str(mean_absolute_error(y_train, predictions)))
print('MAPE: '+str(mean_absolute_percentage_error(y_train, predictions)))

print('-------------------')


test_pred = reg.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))


GRADIENT BOOSTING REGRESSION
RMSE: 1.4085006065370824
R Squared: 0.9771172309190482
MSE : 1.983873958615329
MAE : 0.9935352251162074
MAPE: 0.17917003212808258
-------------------
RMSE: 8.317783919090752
R Squared: 0.15316437878638034
MSE TEST: 69.18552932468471
MAE TEST: 6.239568070180534
MAPE: 0.7352778300900868


In [6]:
new = 72259

In [7]:
7322001

7322001